## 8.5 Implementation of Recurrent Neural Networks from Scratch

In [ ]:
pip install d2l==0.17.2

In [2]:
%matplotlib inline
import math
import torch
from torch import nn
from torch.nn import functional as F
from d2l import torch as d2l

In [3]:
batch_size, num_steps = 32, 35
train_iter, vocab = d2l.load_data_time_machine(batch_size, num_steps)

In [4]:
# One-Hot Encoding
F.one_hot(torch.tensor([0, 2]), len(vocab))

tensor([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0],
        [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0]])

In [5]:
X = torch.arange(10).reshape((2, 5))
F.one_hot(X.T, 28).shape

torch.Size([5, 2, 28])

In [6]:
# Initializing the Model Parameters
def get_params(vocab_size, num_hiddens, device):
  num_inputs = num_outputs = vocab_size

  def normal(shape):
    return torch.randn(size = shape, device = device) * 0.01
  
  # Hidden layer parameters
  W_xh = normal((num_inputs, num_hiddens))
  W_hh = normal((num_hiddens, num_hiddens))
  b_h = torch.zeros(num_hiddens, device = device)
  # Output layer parameters
  W_hq = normal((num_hiddens, num_outputs))
  b_q = torch.zeros(num_outputs, device = device)
  # Attach gradients
  params = [W_xh, W_hh, b_h, W_hq, b_q]
  for param in params:
    param.requires_grad_(True)
  return params

In [7]:
def init_rnn_state(batch_size, num_hiddens, device):
  return (torch.zeros((batch_size, num_hiddens), device = device), )

In [8]:
def rnn(inputs, state, params):
  # Here `inputs` shape: (`num_steps`, `batch_size`, `vocab_size`)
  W_xh, W_hh, b_h, W_hq, b_q = params
  H, = state
  outputs = []
  # Shape of `X` : (`batch_size`, `vocab_size`)
  for X in inputs:
    H = torch.tanh(torch.mm(X, W_xh) + torch.mm(H, W_hh) + b_h)
    Y = torch.mm(H, W_hq) + b_q
    outputs.append(Y)
  return torch.cat(outputs, dim = 0), (H, )

In [9]:
class RNNModelScratch:
  """A RNN Model implemented from scratch."""
  def __init__(self, vocab_size, num_hiddens, device, get_params, init_state, forward_fn):
    self.vocab_size, self.num_hiddens = vocab_size, num_hiddens
    self.params = get_params(vocab_size, num_hiddens, device)
    self.init_state, self.forward_fn = init_state, forward_fn
  
  def __call__(self, X, state):
    X = F.one_hot(X.T, self.vocab_size).type(torch.float32)
    return self.forward_fn(X, state, self.params)
  
  def begin_state(self, batch_size, device):
    return self.init_state(batch_size, self.num_hiddens, device)

In [10]:
num_hiddens = 512
net = RNNModelScratch(len(vocab), num_hiddens, d2l.try_gpu(), get_params, init_rnn_state, rnn)
state = net.begin_state(X.shape[0], d2l.try_gpu())

In [11]:
Y, new_state = net(X.to(d2l.try_gpu()), state)
Y.shape, len(new_state), new_state[0].shape

(torch.Size([10, 28]), 1, torch.Size([2, 512]))

In [12]:
def predict_ch8(prefix, num_preds, net, vocab, device):
  """Generate new characters following the `prefix`."""
  state = net.begin_state(batch_size = 1, device = device)
  outputs = [vocab[prefix[0]]]
  get_input = lambda: torch.tensor([outputs[-1]], device = device).reshape((1,1))
  for y in prefix[1:]: # Warm-up period
    _, state = net(get_input(), state)
    outputs.append(vocab[y])
  for _ in range(num_preds): # Predict `num_preds` steps
    y, state = net(get_input(), state)
    outputs.append(int(y.argmax(dim=1).reshape(1)))
  return ''.join([vocab.idx_to_token[i] for i in outputs])

In [13]:
predict_ch8('time traveller ', 10, net, vocab, d2l.try_gpu())

'time traveller jfkkr<unk>ym j'

In [14]:
def grad_clipping(net, theta):
  """Clip the gradient."""
  if isinstance(net, nn.Module):
    params = [p for p in net.parameters() if p.requires_grad]
  else:
    params = net.params
  norm = torch.sqrt(sum(torch.sum((p.grad ** 2)) for p in params))
  if norm > theta:
    for param in params:
      param.grad[:] *= theta / norm

In [15]:
def train_epoch_ch8(net, train_iter, loss, updater, device, use_random_iter):
  """Train a net within one epoch (defined in Chapter 8)."""
  state, timer = None, d2l.Timer()
  metric = d2l.Accumulator(2) # Sum of training loss, no. of tokens
  for X, Y in train_iter:
    if state is None or use_random_iter:
      # Initialize `state` when either it is the first iteration or using random sampling
      state = net.begin_state(batch_size = X.shape[0], device = device)
    else:
      if isinstance(net, nn.Module) and not isinstance(state, tuple):
        # `state` is a tensor for `nn.GRU`
        state.detach_()
      else:
        # `state` is a tuple of tensors for `nn.LSTM` and for our custom scratch implementation
        for s in state:
          s.detach_()
    y = Y.T.reshape(-1)
    X, y = X.to(device), y.to(device)
    y_hat, state = net(X, state)
    l = loss(y_hat, y.long()).mean()
    if isinstance(updater, torch.optim.Optimizer):
      updater.zero_grad()
      l.backward()
      grad_clipping(net, 1)
      updater.step()
    else:
      l.backward()
      grad_clipping(net, 1)
      # Since the `mean` function has been invoked
      updater(batch_size = 1)
    metric.add(l * y.numel(), y.numel())
  return math.exp(metric[0] / metric[1]), metric[1] / timer.stop()

In [16]:
def train_ch8(net, train_iter, vocab, lr, num_epochs, device, use_random_iter = False):
  """Train a model (defined in Chapter 8)."""
  loss = nn.CrossEntropyLoss()
  animator = d2l.Animator(xlabel = 'epoch', ylabel = 'perplexity', legend = ['train'], xlim = [10, num_epochs])
  # Initialize
  if isinstance(net, nn.Module):
    updater = torch.optim.SGD(net.parameters(), lr)
  else:
    updater = lambda batch_size: d2l.sgd(net.params, lr, batch_size)
  predict = lambda prefix: predict_ch8(prefix, 50, net, vocab, device)
  # Train and predict
  for epoch in range(num_epochs):
    ppl, speed = train_epoch_ch8(net, train_iter, loss, updater, device, use_random_iter)
    if (epoch + 1) % 10 == 0:
      print(predict('time traveller'))
      animator.add(epoch + 1, [ppl])
  
  print(f'perplexity {ppl:.1f}, {speed:.1f} tokens/sec on {str(device)}')
  print(predict('time traveller'))
  print(predict('traveller'))

In [17]:
num_epochs, lr  = 500, 1
train_ch8(net, train_iter, vocab, lr, num_epochs, d2l.try_gpu())

perplexity 1.1, 71809.4 tokens/sec on cuda:0
time traveller held in his hand was a glitteringmetallic framewo
travelleryou can show black is white by argument said filby


ImportError: ignored

<Figure size 252x180 with 1 Axes>

In [18]:
net = RNNModelScratch(len(vocab), num_hiddens, d2l.try_gpu(), get_params, init_rnn_state, rnn)
train_ch8(net, train_iter, vocab, lr, num_epochs, d2l.try_gpu(), use_random_iter = True)

perplexity 1.4, 65787.8 tokens/sec on cuda:0
time travellerit s against reason said filbywan afcube that dles
traveller ffrce dimensions and the latter becand of s ace t


ImportError: ignored

<Figure size 252x180 with 1 Axes>

## Summary
* We can train an RNN-based character-level language model to generate text following the user-provided text prefix.

* A simple RNN language model consists of input encoding, RNN modeling, and output generation.

* RNN models need state initialization for training, through random sampling and sequential partitioning use different ways.

* When using sequential partitioning, we need to detach the gradient to reduce computational cost.

* A warm-up period allows a model to update itself(e.g, obtain a better hidden state than its initialized value) before making any prediction.

* Gradient clipping prevents gradient explosion, but it cannot fix vanishing gradients.

## 8.6 Concise Implementation of Recurrent Neural Networks

In [ ]:
import torch
from torch import nn
from torch.nn import functional as F
from d2l import torch as d2l

In [19]:
batch_size, num_steps = 32, 35
train_iter, vocab = d2l.load_data_time_machine(batch_size, num_steps)

In [20]:
num_hiddens = 256
rnn_layer = nn.RNN(len(vocab), num_hiddens)

In [21]:
state = torch.zeros((1, batch_size, num_hiddens))
state.shape

torch.Size([1, 32, 256])

In [22]:
X = torch.rand(size = (num_steps, batch_size, len(vocab)))
Y, state_new = rnn_layer(X, state)
Y.shape, state_new.shape

(torch.Size([35, 32, 256]), torch.Size([1, 32, 256]))

In [23]:
class RNNModel(nn.Module):
  """The RNN model."""
  def __init__(self, rnn_layer, vocab_size, **kwargs):
    super(RNNModel, self).__init__(**kwargs)
    self.rnn = rnn_layer
    self.vocab_size = vocab_size
    self.num_hiddens = self.rnn.hidden_size
    # If the RNN is bidirectional (to be introduced later), `num_directions` should be 2, else it should be 1.
    if not self.rnn.bidirectional:
      self.num_directions = 1
      self.linear = nn.Linear(self.num_hiddens, self.vocab_size)
    else:
      self.num_directions = 2
      self.linear = nn.Linear(self.num_hiddens * 2 ,self.vocab_size)
  
  def forward(self, inputs, state):
    X = F.one_hot(inputs.T.long(), self.vocab_size)
    X = X.to(torch.float32)
    Y, state = self.rnn(X, state)
    # The fully connected layer will first change the shape of `Y` to (`num_steps` * `batch_size`, `num_hiddens`).
    # Its output shape is (`num_steps` * `batch_size`, `vocab_size`).
    output = self.linear(Y.reshape((-1, Y.shape[-1])))
    return output, state
  
  def begin_state(self, device, batch_size = 1):
    if not isinstance(self.rnn, nn.LSTM):
      # `nn.GRU` takes a tensor as hidden state
      return torch.zeros((self.num_directions * self.rnn.num_layers, batch_size, self.num_hiddens), device = device)
    else:
      # `nn.LSTM` takes a tuple of hidden states
      return (torch.zeros((
          self.num_directions * self.rnn.num_layers, batch_size, self.num_hiddens), device = device),
          torch.zeros((
              self.num_directions * self.rnn.num_layers, batch_size, self.num_hiddens), device = device))

In [24]:
device = d2l.try_gpu()
net = RNNModel(rnn_layer, vocab_size = len(vocab))
net = net.to(device)
d2l.predict_ch8('time traveller', 10, net, vocab, device)

'time travellercf f<unk>f<unk>f<unk>f'

In [25]:
num_epochs, lr = 500, 1
d2l.train_ch8(net, train_iter, vocab, lr, num_epochs, device)

perplexity 1.3, 107581.4 tokens/sec on cuda:0
time traveller for sopeatificelutre or the three dimensions they
travellerepercallysood mink of the in any dimentions it mas


ImportError: ignored

<Figure size 252x180 with 1 Axes>

## Summary
* High-level APIs of the deep learning framework provides an implementation of the RNN layer.

* The RNN layer of high-level APIs returns an output and an updated hidden state, where the output does not involve output layer computation.

* Using high-level APIs leads to faster RNN training than using its implementation from scratch.

## 8.7 Backpropagation Through Time

## Summary
* Backpropagation through time is merely an application of backpropagation to sequence models with a hidden state.

* Truncation is needed for computational convenience and numerical stability, such as regular truncation and randomized truncation.

* High powers of matrices can lead to divergent or vanishing eigenvalues. This manifests itself in the form of exploding or vanishing gradients.

* For efficient computation, intermediate values are cached during backpropagation through time.